Group Name: AG xx.

Student Name (Student ID):

1. xxxx xxxxx (xxxxxxx)

2. xxxx xxxxx (xxxxxxx)

3. xxxx xxxxx (xxxxxxx)

# Question 1

Consider the maze shown below. The Maze has 16 rows and 24 columns The objective is to find a shortest path from cell $S$ to cell $G$.


![Maze](Maze_Assignment_1.jpg)


The agent can take four actions in each cell: 'RIGHT', 'DOWN', 'UP', 'LEFT'.  

Each cell is represented as $(x,y)$, where $x$ indicates row number and $y$ indicates column number. Action 'UP' takes the agent from cell $(x,y)$ to $(x+1,y)$. Action 'DOWN' takes the agent from cell $(x,y)$ to $(x-1,y)$. Action 'RIGHT' takes the agent from cell $(x,y)$ to $(x,y+1)$. Action 'LEFT' takes the agent from cell $(x,y)$ to $(x,y-1)$. The triplet $(s,a,s')$  indicates that taking action $a$ at state $s$ leads to state $s'$. Actions 'LEFT' or 'RIGHT' cost 10 units for all $(s,a,s')$. Actions 'UP' or 'DOWN' cost 1 unit for all  $(s,a,s')$.  The agent cannot move into cells that are shaded. Assume that the agent knows the boundaries of the maze and has full observability. Consequently, at the bottom (row 0) and top (row 15), the agent will not take actions 'DOWN' and 'UP', respectively; at left (column 0) and right (column 23) columns, the agent will not take 'LEFT' and 'RIGHT' actions, respectively. Similalry, the agent will not take actions that lead to shaded region in the maze.

## **Q1.a: Class Maze(Problem)** [3 Marks]

Write a Maze class to create a model for this problem. You should not use an explicit state space model. The modelling should inherit the abstract class 'Problem' (given below). With the problem formulation, find the shortest path from S to G cell. Propose and implement multiple heuristics (at least two heuristics) for informed search algorithms. 

### Blocked Rows and Columns:
[(14,9), (13,10), (12,10), (11,10), (10,10), (10,9), (9,9), (8,9), (7,9), (6,9), (11,12), (10,12), (11,13), (10,13)]

### Goal Loc:
(11,9)

### Init Loc:
(8,10)

### *All tuple written (row, col)

## **Q1.b: Analysis of the Algorithms** [7 Marks]

1. Solve the above Maze problem using the following algorithms

    a. Breadth-First Search

    b. Depth-First Search with Cycle-Check

    c. Iterative-Deepening Search with Cycle-Check

    d. Uniform-Cost Search

    e. A* Search 

    f. Greedy Best-first Search

    g. Any other variants for search algorithms that are not discussed in the class (bonus/optional question) 

2. Identify the number of nodes generated, number of nodes expanded, maximum frontier size, and path-cost for the above algorithms. 
 
3. Compare the performance of informed search algorithms with proposed heuristics. Identify the best performing heuristic and explain.
 
4. Draw a bar plot comparing the statistics of the algorithms and explain the results. 

Note 1: You must follow the problem formulation discussed in the class. A abstract class for Problem amd Node definition is presented below. The search tree generation should follow the template discussed in the class (i.e., Node class, expand methods, etc.). 

Note 2: If you are borrowing a block of code (for example, helper functions or data structures, etc.) from AIMA4e repository, you have to acknowledge it in the code. 

Note 3: The code should be written in a single jupyter notebook file.

In [1]:

class Problem:
    """The abstract class for a formal problem. A new domain subclasses this,
    overriding `actions` and `results`, and perhaps other methods.
    The default heuristic is 0 and the default action cost is 1 for all states.
    When you create an instance of a subclass, specify `initial`, and `goal` states 
    (or give an `is_goal` method) and perhaps other keyword args for the subclass."""

    def __init__(self, initial=None, goal=None, **kwds): 
        self.__dict__.update(initial=initial, goal=goal, **kwds) 
        
    def actions(self, state):        raise NotImplementedError
    def result(self, state, action): raise NotImplementedError
    def is_goal(self, state):        return state == self.goal
    def action_cost(self, s, a, s1): return 1
    def h(self, node):               return 0
    
    def __str__(self):
        return '{}({!r}, {!r})'.format(
            type(self).__name__, self.initial, self.goal)

In [2]:
# Use the following Node class to generate search tree
import math
class Node:
    "A Node in a search tree."
    def __init__(self, state, parent=None, action=None, path_cost=0):
        self.__dict__.update(state=state, parent=parent, action=action, path_cost=path_cost)

    def __repr__(self): return '<{}>'.format(self.state)
    def __len__(self): return 0 if self.parent is None else (1 + len(self.parent))
    def __lt__(self, other): return self.path_cost < other.path_cost 


In [4]:
from pprint import pprint
class Maze(Problem):
    #Your code goes here
    def __init__(self, rows, columns, initial=None, goal=None, **kwds):
        super().__init__(initial, goal, **kwds)
        self.cols = columns
        self.rows = rows
        self.blockers = {}
        
    def add_blocker(self,blocker):
        r,c = blocker
        if(r in self.blockers):
            self.blockers[r].append(c)
        else:
            self.blockers[r] = [c]
            
    def print_blockers(self):
        pprint(self.blockers)
        
    def actions(self, action, state): 
        r,c = state
        cost = 0
        if action == 'up':
            r += 1
            cost = 1
        if action == 'down':
            r -= 1
            cost = 1
        if action == 'left':
            c -= 1
            cost = 10
        if action == 'right':
            c += 1
            cost = 10
        if(r < 0 or c < 0 or r > self.rows or c > self.cols ):
            return {'state': state, 'cost': 0}
        if(r in self.blockers and c in self.blockers[r]):
            return {'state': state, 'cost': 0}
        return {'state':(r,c), 'cost': cost}
    

In [5]:
from pprint import pprint
maze = Maze(15,23, (8,10),(11,9))
blocker = [(14,9), (13,10), (12,10), (11,10), (10,10), (10,9), (9,9), (8,9), (7,9), (6,9), (11,12), (10,12), (11,13), (10,13)]

for b in blocker:
    maze.add_blocker(b)
    
pprint(maze.actions('right', (14,8)))

{'cost': 0, 'state': (14, 8)}


In [6]:

#Helper function for cycle check borrowed from search4e.ipynb from the AIMA4e repository

def is_cycle(node, k=50):
    "Does this node form a cycle of length k or less?"
    def find_cycle(ancestor, k):
        return (ancestor is not None and k > 0 and
                (ancestor.state == node.state or find_cycle(ancestor.parent, k - 1)))
    return find_cycle(node.parent, k)


In [7]:
### BFS
from collections import deque

def bfs(maze):
    visited_nodes = {}
    actions = ['up', 'down', 'left', 'right']
    current_state = (8,10)
    visited_nodes[current_state[0]] = [current_state[1]]

    frontier_node = Node(current_state)
    count_node = 1
    q = deque([frontier_node])
    frontier_count = len(q)
    max_frontier = 1


    while(len(q) != 0):
        expand_node = q.popleft()
        for action in actions:
            s = maze.actions(action, expand_node.state)
            r = s['state'][0]
            c = s['state'][1]
            if(expand_node.state == s['state'] or (r in visited_nodes and c in visited_nodes[r])): 
                continue
            frontier_node = Node(s['state'], expand_node, action, s['cost'])
            count_node += 1
            if(maze.is_goal(frontier_node.state)):
                print('Goal reached!')
                pprint({'goal_node': frontier_node, 'max_frontier': max_frontier})
                return {'goal_node': frontier_node, 'max_frontier': max_frontier, 'count_node': count_node}
            if(r in visited_nodes):
                visited_nodes[r].append(c)
            else:
                visited_nodes[r] = [c]
            q.append(frontier_node)

        frontier_count = len(q)
        max_frontier = max(max_frontier, frontier_count)
    #Goal not found    
    return False


solution = bfs(maze)

trace_node = solution['goal_node']
max_frontier = solution['max_frontier']
count_node = solution['count_node']
total_cost = 0
path = [trace_node]

while(trace_node.parent != None):
    total_cost += trace_node.path_cost
    trace_node = trace_node.parent
    path.append(trace_node)

print('\n')
print('Total cost:')
print(total_cost, '\n')
    
print('Maximum number of frontiers:')
print(max_frontier, '\n')

print('Number of nodes created:')
print(count_node, '\n')

print('Path taken')
print(path[::-1])

Goal reached!
{'goal_node': <(11, 9)>, 'max_frontier': 30}


Total cost:
39 

Maximum number of frontiers:
30 

Number of nodes created:
198 

Path taken
[<(8, 10)>, <(7, 10)>, <(6, 10)>, <(5, 10)>, <(5, 9)>, <(5, 8)>, <(6, 8)>, <(7, 8)>, <(8, 8)>, <(9, 8)>, <(10, 8)>, <(11, 8)>, <(11, 9)>]


In [13]:
### DFS with Cycle Check


def dfs(maze):

    actions = ['up', 'down', 'left', 'right']
    current_state = (8,10)
    
    frontier_node = Node(current_state)

    count_node = 1
    
    q = deque([frontier_node])

    frontier_count = len(q)
    max_frontier = 1
    max_cutofflimit= 1000
    count_parent= 0

    while(len(q) != 0 and count_parent<max_cutofflimit):
        
        expand_node = q.pop()
        count_parent+=1
        
        if(maze.is_goal(expand_node.state)):
            print('Goal reached!')
            pprint({'goal_node': expand_node, 'max_frontier': max_frontier})
            return {'goal_node': expand_node, 'max_frontier': max_frontier, 'count_node': count_node}

        else:
            if is_cycle(expand_node):
                continue
            
            for action in actions:
                s = maze.actions(action, expand_node.state)

                if(expand_node.state == s['state']): 
                    continue
                
                
                frontier_node = Node(s['state'], expand_node, action, expand_node.path_cost+s['cost'])

                count_node += 1  
                
                q.append(frontier_node)                

        frontier_count = len(q)
        
        max_frontier = max(max_frontier, frontier_count)


    #Goal not found    
    return False

solution_dfs = dfs(maze)

if solution_dfs==False:
    print('No solution found.')
    
else:
    trace_node = solution_dfs['goal_node']
    max_frontier = solution_dfs['max_frontier']
    count_node = solution_dfs['count_node']
    total_cost = trace_node.path_cost
    path = [trace_node]
    
    while(trace_node.parent != None):
        trace_node = trace_node.parent
        path.append(trace_node)

    print('\n')
    print('Total cost:')
    print(total_cost, '\n')
    
    print('Maximum number of frontiers:')
    print(max_frontier, '\n')

    print('Number of nodes created:')
    print(count_node, '\n')

    print('Path taken')
    print(path[::-1])

No solution found.


In [15]:
### IDS with Cycle Check

## DLS
def dls(maze,l):

    actions = ['up', 'down', 'left', 'right']
    current_state = (8,10)
    
    frontier_node = Node(current_state)

    count_node = 1
    
    q = deque([frontier_node])

    frontier_count = len(q)
    max_frontier = 1


    while(len(q) != 0 ):
        
        expand_node = q.pop()
        
        if(maze.is_goal(expand_node.state)):
            print('Goal reached!')
            pprint({'goal_node': expand_node, 'max_frontier': max_frontier})
            return {'goal_node': expand_node, 'max_frontier': max_frontier, 'count_node': count_node}

        else:

            if len(expand_node)>l:
                continue
            
            if is_cycle(expand_node):
                continue
            
            for action in actions:
                s = maze.actions(action, expand_node.state)

                if(expand_node.state == s['state']): 
                    continue
                
                frontier_node = Node(s['state'], expand_node, action, expand_node.path_cost+s['cost'])
                    
                count_node += 1                 
                q.append(frontier_node)                

        frontier_count = len(q)        
        max_frontier = max(max_frontier, frontier_count)


    #Goal not found    
    return False

## IDS
def ids(maze):
    for i in range(0,99999):
        solution_dls=dls(maze,i)

        if solution_dls==False:
            print(f'Limit l={i}, no solution found. Continue with l={i+1}')
            
        else:
            print(f'Solution found with l={i}')
            return solution_dls
            
    
solution_ids=ids(maze)

trace_node = solution_ids['goal_node']
max_frontier = solution_ids['max_frontier']
count_node = solution_ids['count_node']
total_cost = trace_node.path_cost

path = [trace_node]
    
while(trace_node.parent != None):
    trace_node = trace_node.parent
    path.append(trace_node)

print('\n')
print('Total cost:')
print(total_cost, '\n')
    
print('Maximum number of frontiers:')
print(max_frontier, '\n')

print('Number of nodes created:')
print(count_node, '\n')

print('Path taken')
print(path[::-1])


Limit l=0, no solution found. Continue with l=1
Limit l=1, no solution found. Continue with l=2
Limit l=2, no solution found. Continue with l=3
Limit l=3, no solution found. Continue with l=4
Limit l=4, no solution found. Continue with l=5
Limit l=5, no solution found. Continue with l=6
Limit l=6, no solution found. Continue with l=7
Limit l=7, no solution found. Continue with l=8
Limit l=8, no solution found. Continue with l=9
Limit l=9, no solution found. Continue with l=10
Limit l=10, no solution found. Continue with l=11
Goal reached!
{'goal_node': <(11, 9)>, 'max_frontier': 36}
Solution found with l=11


Total cost:
39 

Maximum number of frontiers:
36 

Number of nodes created:
153887 

Path taken
[<(8, 10)>, <(7, 10)>, <(6, 10)>, <(5, 10)>, <(5, 9)>, <(5, 8)>, <(6, 8)>, <(7, 8)>, <(8, 8)>, <(9, 8)>, <(10, 8)>, <(11, 8)>, <(11, 9)>]


In [17]:
### Uniform Cost Search

from queue import PriorityQueue


def bestfs(maze,f):
    
    actions = ['up', 'left', 'down', 'right']
    current_state = (8,10)

    frontier_node = Node(current_state)
    
    #store visited notes in dictionary
    visited_nodes = {current_state:frontier_node}

    count_node = 1
    q = PriorityQueue()
    
    if f=='g':
        q.put((0, [frontier_node]))
    
    frontier_count = q.qsize()
    max_frontier = 1

    
    #testing
    '''
    print("Initial Frontier")
    for elem in q.queue:
        print(elem)
    print("Initial Reached")
    print(visited_nodes)
    mycount=0
    '''
    
    while(q.qsize() != 0):
        expand_node = q.get()[1][0]

        #testing
        '''
        print("EXPANDING PARENT Node")
        print(expand_node)
        '''  
        
        if(maze.is_goal(expand_node.state)):
            print('Goal reached!')
            pprint({'goal_node': expand_node, 'max_frontier': max_frontier})
            return {'goal_node': expand_node, 'max_frontier': max_frontier, 'count_node': count_node}
          
        
        for action in actions:
            s = maze.actions(action, expand_node.state)

            
            if (expand_node.state == s['state']): 
                continue

            if (visited_nodes.get(s['state'],0) != 0):
                previous_reached_node=visited_nodes[s['state']]    

                if expand_node.path_cost+s['cost']>=previous_reached_node.path_cost:
                    continue
                              
            frontier_node = Node(s['state'], expand_node, action, expand_node.path_cost+s['cost'])

            count_node += 1
    
            visited_nodes[s['state']]=frontier_node
            
            if f=='g':
                q.put((-(frontier_node.path_cost), [frontier_node]))            

            #testing
            '''
            print("Frontier")
            for elem in q.queue:
                print(elem)
            print("Reached")
            print(visited_nodes)
            '''
        
        frontier_count = q.qsize()
        max_frontier = max(max_frontier, frontier_count)
        
        '''
        mycount+=1
        if mycount==999999:
            break
        '''
    #Goal not found    
    return False


solution_ucs = bestfs(maze,'g')


trace_node = solution_ucs['goal_node']
max_frontier = solution_ucs['max_frontier']
count_node = solution_ucs['count_node']
total_cost = trace_node.path_cost

path = [trace_node]
    
while(trace_node.parent != None):
    trace_node = trace_node.parent
    path.append(trace_node)

print('\n')
print('Total cost:')
print(total_cost, '\n')
    
print('Maximum number of frontiers:')
print(max_frontier, '\n')

print('Number of nodes created:')
print(count_node, '\n')

print('Path taken')
print(path[::-1])


Goal reached!
{'goal_node': <(11, 9)>, 'max_frontier': 61}


Total cost:
461 

Maximum number of frontiers:
61 

Number of nodes created:
117 

Path taken
[<(8, 10)>, <(8, 11)>, <(8, 12)>, <(8, 13)>, <(8, 14)>, <(8, 15)>, <(8, 16)>, <(8, 17)>, <(8, 18)>, <(8, 19)>, <(8, 20)>, <(8, 21)>, <(8, 22)>, <(8, 23)>, <(9, 23)>, <(10, 23)>, <(10, 22)>, <(10, 21)>, <(10, 20)>, <(10, 19)>, <(10, 18)>, <(10, 17)>, <(10, 16)>, <(10, 15)>, <(10, 14)>, <(11, 14)>, <(12, 14)>, <(12, 13)>, <(12, 12)>, <(12, 11)>, <(13, 11)>, <(14, 11)>, <(14, 10)>, <(15, 10)>, <(15, 9)>, <(15, 8)>, <(15, 7)>, <(15, 6)>, <(15, 5)>, <(15, 4)>, <(15, 3)>, <(15, 2)>, <(15, 1)>, <(15, 0)>, <(14, 0)>, <(13, 0)>, <(13, 1)>, <(13, 2)>, <(13, 3)>, <(13, 4)>, <(13, 5)>, <(13, 6)>, <(13, 7)>, <(13, 8)>, <(13, 9)>, <(12, 9)>, <(11, 9)>]
